In [1]:
from bs4 import BeautifulSoup
import math
import pandas as pd
final_list=[]#final list to be the df
import json
import requests

In [7]:
final_list=[]
url = 'https://uk.trustpilot.com/review/idealboilers.com'
#making a request to get the number of reviews
r=requests.get(url)
soup = BeautifulSoup(r.text, 'lxml')
review_count_h2=soup.find('h2',class_="header--inline").text
review_count_h2=review_count_h2.strip().split(' ')[0].strip()
review_count=int(review_count_h2.replace(',', ''))
#there are 20 reviews per page so pages can be calculated as
pages=int(math.ceil(review_count/20))
#change range to 1 to pages+1
for pg in range(1, pages+1):
  pg = url + '?page=' + str(pg)
  r=requests.get(pg)
  soup = BeautifulSoup(r.text, 'lxml')
  for paragraph in soup.find_all('section',class_='review__content'):
     try:
         title=paragraph.find('h2',class_='review-content__title').text.strip()
         content=paragraph.find('p',class_='review-content__text').text.strip()
         datedata= json.loads(paragraph.find('div',class_='review-content-header__dates').text)
         date=datedata['publishedDate'].split('T')[0]
         rating_class=paragraph.find('div',class_='star-rating')['class']
         rating=rating_class[1].split('-')[-1]
         star = paragraph.find("div", {"class":"star-rating star-rating--medium"}).find('img').get('alt')
         final_list.append([title,content,date,rating,star])
     except AttributeError:
        pass
df = pd.DataFrame(final_list,columns=['Title','Content','Date','Rating','Star'])

In [8]:
df.head()

,Title,Content,Date,Rating,Star
0,Stopped installing Ideal Boilers.,Carbon credits are not being paid within a rea...,2020-03-29,medium,1 star: Bad
1,Always have great back up from Ideal…,Always have great back up from Ideal Boilers,2020-03-29,medium,5 stars: Excellent
2,Solid and trustworthy boilers,Bought a house that already had an Ideal boile...,2020-03-28,medium,5 stars: Excellent
3,would give ideal boilers and the…,would give ideal boilers and the service plans...,2020-03-26,medium,1 star: Bad
4,Great value boilers,"Great value boilers, with industry leading war...",2020-03-24,medium,5 stars: Excellent


In [9]:
df.to_csv('Ideal Boilers Data New.csv', index=False) 